[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Divyanshnahar/cs267-lab-report/blob/main/LAB-2/plagiarism_Astar.ipynb)

In [ ]:
import re
import heapq
from nltk.tokenize import sent_tokenize

# -----------------------------
# TEXT PREPROCESSING
# -----------------------------
def preprocess(text):
    """Convert text to lowercase, remove punctuation, and split into sentences."""
    sentences = sent_tokenize(text.lower())
    sentences = [re.sub(r'[^\w\s]', '', s).strip() for s in sentences if s.strip()]
    return sentences


# -----------------------------
# COST FUNCTION
# -----------------------------
def cost_function(a, b):
    """Sentence matching cost with sharper penalty for dissimilar text."""
    if a == b:
        return 0.0

    # Token-based difference instead of per-character
    a_tokens = a.split()
    b_tokens = b.split()
    overlap = len(set(a_tokens) & set(b_tokens))
    total = max(len(a_tokens), len(b_tokens), 1)

    similarity = overlap / total
    cost = (1 - similarity) * 10.0  # strong penalty for low overlap
    return cost


# -----------------------------
# A* SEARCH ALIGNMENT
# -----------------------------
def a_star_search(doc1_sents, doc2_sents):
    """Find minimal alignment cost between two sets of sentences."""
    n, m = len(doc1_sents), len(doc2_sents)
    pq = [(0, 0, 0)]  # (cost, i, j)
    visited = set()

    while pq:
        cost, i, j = heapq.heappop(pq)
        if (i, j) in visited:
            continue
        visited.add((i, j))

        if i == n and j == m:
            return cost

        if i < n and j < m:
            heapq.heappush(pq, (cost + cost_function(doc1_sents[i], doc2_sents[j]), i + 1, j + 1))
        if i < n:
            heapq.heappush(pq, (cost + 8.0, i + 1, j))
        if j < m:
            heapq.heappush(pq, (cost + 8.0, i, j + 1))

    return float("inf")


# -----------------------------
# PLAGIARISM DETECTION
# -----------------------------
def detect_plagiarism(doc1, doc2):
    """Detect plagiarism using alignment cost and similarity score."""
    doc1_sents = preprocess(doc1)
    doc2_sents = preprocess(doc2)

    alignment_cost = a_star_search(doc1_sents, doc2_sents)

    total_len = max(len(" ".join(doc1_sents)), len(" ".join(doc2_sents)), 1)
    similarity = max(0, 1 - alignment_cost / (total_len / 3.5))  # stronger normalization

    if similarity >= 0.9:
        label = "⚠️ Potential plagiarism detected"
    elif similarity >= 0.6:
        label = "⚠️ Partial similarity detected"
    else:
        label = "✅ No plagiarism detected"

    print(f"{label} (Alignment Cost: {alignment_cost:.2f}, Similarity: {similarity*100:.2f}%)")


# -----------------------------
# TEST CASES
# -----------------------------
if __name__ == "__main__":
    print("\n--- Test Case 1: Identical Documents ---")
    detect_plagiarism(
        "Artificial Intelligence is the study of intelligent agents.",
        "Artificial Intelligence is the study of intelligent agents."
    )

    print("\n--- Test Case 2: Slightly Modified Documents ---")
    detect_plagiarism(
        "Artificial Intelligence is the study of intelligent agents.",
        "AI is the study of smart agents."
    )

    print("\n--- Test Case 3: Totally Different Documents ---")
    detect_plagiarism(
        "The sun rises in the east and sets in the west.",
        "Machine learning algorithms improve through experience."
    )

    print("\n--- Test Case 4: Partially Overlapped Documents ---")
    detect_plagiarism(
        "Artificial Intelligence techniques are applied in robotics and healthcare.",
        "Robotics and healthcare use Artificial Intelligence for automation."
    )
